In [ ]:
import glob
import os
if os.path.exists('Y:/VIDEOS/1119/Videos/'):
    print('Y:/VIDEOS/1119/Videos/ exists')

## Validation of Folders/Paths

In [ ]:
import glob
import os
folder = 'Y:/VIDEOS'

ids = os.listdir(folder)
excluded_list = []
for idx in ids:
    
    # check folder name
    if os.path.exists(folder + '/' + idx + '/Videos/'):
        print(f'idx: {idx}, folder: {folder + "/" + idx + "/Videos/"}')
        files = glob.glob(folder + '/' + idx + '/Videos/*')
    else:
        # print(f'idx: {idx}, folder: {folder + "/" + idx + "/Video/"}')
        files = glob.glob(folder + '/' + idx + '/**/*.asf')    

    # check for 827 or 984
    if os.path.exists(folder + '/' + idx + '/827000/') or os.path.exists(folder + '/' + idx + '/984/'):
        print(f'idx: {idx}, 827 or 984')
        excluded_list.append(idx)
        continue
    
    # check for 827000 or 984 in nested folder
    if os.path.exists(folder + '/' + idx + '/Video/827000/') or os.path.exists(folder + '/' + idx + '/Video/984/'):
        print(f'idx: {idx}, 827 or 984')
        excluded_list.append(idx)
        continue 

    if len(files) < 1:
        print(f'idx: {idx}, files: {len(files)}')
        excluded_list.append(idx)



### Validation for Empty Video Folder

In [ ]:
import glob
import os
folder = 'Y:/VIDEOS'
# video_list = glob.glob(folder + '/*/Video/**/*.asf')
# video_list

ids = os.listdir(folder)
excluded_list = []
for idx in ids:
    files = glob.glob(folder + '/' + idx + '/Video/**/*.asf')
    if len(files) < 1:
        print(f'idx: {idx}, files: {len(files)}')
        excluded_list.append(idx)

In [ ]:
excluded_list = ['1009 1010',
                 '1011 1012',
                 '1003 1004',
                 '1031 1017',
                 '1011 1012',
                 '1043 1044',
                 '1055 1056',
                 '1074 1075',
                 '1099 1100',
                 '1013 1014',
                 '1003 1004-nonAI',
                 '1005-nonAI',
                 '1073',
                 '1082',
                 '1094',
                 '1097',
                 '2002',
                 '2062'
                 ]


In [ ]:
import os
import json
import random
import metaData
import argparse
from tqdm import tqdm
import datetime
import pandas as pd


In [ ]:
json_file = 'Datafiles/data_storage_Jan2023.json'
with open(json_file) as file:
    data = json.load(file)

In [ ]:
days_list = {}
days = []
for i, d in enumerate(data['data'][:500]):
    if d['id'] in days_list.keys():
        days_list[d['id']].append(d['day'])
    elif d['id'] not in days_list.keys():
        days_list[d['id']] = list()
        days_list[d['id']].append(d['day'])
    # print(f"Index: {i}, Data: {d['day']}")
    # days.append(d['day'])

days_list_sorted = {k: sorted(v) for k, v in days_list.items()}

In [ ]:
df = pd.read_json(json_file)
dt = pd.DataFrame(df['data'], columns=['id', 'day', 'duration'])


jn = pd.json_normalize(df['data'])


In [ ]:
col_list = []
ts_list = []
for col, row in jn.iloc[:10].iterrows():
    for idx in row:
        # col_list.append(idx)
        if type(idx) == list and len(idx) > 0:
            for item in idx:
                ts_list.append(item)
        else:
            col_list.append(idx)
            if ts_list != []:
                col_list.append(ts_list)
                ts_list = []
            

In [ ]:
jn.columns

ts_cols = ['yawn.timestamp',
           'smoking.timestamp',
           'mobilephone.timestamp',
           'distraction.timestamp',
           'eyeclosing.timestamp',
           'crossinglane.timestamp',
           'nearcollision.timestamp',
           'stopsign.timestamp',
           'redlight.timestamp',
           'pedestrian.timestamp',
           ]


### Data Aggregate

In [ ]:
#  sort a list of dictionaries by value1 and value2
def sort_list_of_dict_by_value(list_of_dict, value1, value2):
    return sorted(list_of_dict, key=lambda k: (k[value1], k[value2]))


# get next 90 days from a given date
# return a single date for next 90 days
def get_next_90_days(start_date):
    yyyy = start_date[:4]
    mm = start_date[4:6]
    dd = start_date[6:]

    converted_date = datetime.datetime(int(yyyy), int(mm), int(dd))
    next_months = converted_date + datetime.timedelta(days=90)
    return str(next_months)[:10].replace('-', '')

def get_next_90_days_data_from_sorted_list_of_dict(list_of_dict, value1, value2):
    next_90_days = []
    for i, d in enumerate(list_of_dict):
        if i == len(list_of_dict)-1:
            break
        next_90_days.append(get_next_90_days(d[value2]))
    return next_90_days

# calculate number of day between two dates
def days_between(d1, d2):
    d1 = datetime.datetime.strptime(d1, "%Y%m%d")
    d2 = datetime.datetime.strptime(d2, "%Y%m%d")
    return abs((d2 - d1).days)

# days_between('20200301', '20200102')

In [ ]:
sl = sort_list_of_dict_by_value(data['data'], 'id', 'day')
d = get_next_90_days_data_from_sorted_list_of_dict(sl, 'id', 'day')

### Extracting Data Based on Timestamp

In [ ]:
# filePath = 'Z:/VIDEOS/1010/Disk_files/debug/'
def extract_indices_from_log(filePath,file):
    alarmsDict = {'NOBODY': 0, 'LOOKING_DOWN': 0, 'SMOKING': 0, 'CALLING':0, 'LDW': 0, 'EYE_CLOSED': 0, 'LDW_R': 0, 'LDW_L':0, 'FCW':0, 'camera cover!':0, 'infrared block!': 0 }
    alarmsTimeStamp = {'NOBODY': [], 'LOOKING_DOWN': [], 'SMOKING': [], 'CALLING':[], 'LDW': [], 'EYE_CLOSED': [], 'LDW_R': [], 'LDW_L':[], 'FCW':[], 'camera cover!':[], 'infrared block!': [] }
    # alarmsDict = {'alarm_type 5': 0, 'alarm_type 4': 0, 'alarm_type 3': 0, 'alarm_type 1': 0, 'alarm_type 2': 0, 'alarm_type 17': 0, 'alarm_type 18': 0, 'alarm_type 27':0, 'alarm_type 16':0, 'alarm_type 9':0, 'alarm_type 7': 0 }
    # alarmsTimeStamp = {'alarm_type 5': [], 'alarm_type 4': [], 'alarm_type 3': [], 'alarm_type 1': [], 'alarm_type 2': [], 'alarm_type 17': [], 'alarm_type 18': [], 'alarm_type 27':[], 'alarm_type 16':[], 'alarm_type 9':[], 'alarm_type 7': [] }
    
    error_files = []
    # print(f"Processing debug file {file}")
    with open(os.path.join(filePath, file), 'r') as logFile:
        lines = logFile.read().splitlines()
        for line in lines:
            words = line.split(" ")
            """
            Retrieve alarm will be here
            """
            for alarm in alarmsDict.keys():
                if alarm in words:
                    alarmsDict[alarm] += 1
                    # below is date extraction so needs to be corrected****** --> currently is this -->[20210504-140826][03][baseIVS]
                    try:
                        timestamp = "".join(words[0].split('[')[1].split('-')).strip(']')
                        alarmsTimeStamp[alarm].append(timestamp)
                    except Exception:
                        error_files.append(file)

        # logFile.close()
        # print(alarmsDict)
    return alarmsDict, alarmsTimeStamp, error_files

In [ ]:
excluded_list = ['1003 1004','1031 1017','1011 1012','1043 1044','1013 1014','1003 1004-nonAI', '1005-nonAI', '1073', '2062']
ids = []
for item in os.listdir('Z:/VIDEOS/'):
    if item not in excluded_list:
        ids.append(item)

In [ ]:
jsonfile = []
mapping = {'NOBODY': 0,
           'LOOKING_DOWN': 1,
           'SMOKING': 2, 
           'CALLING': 3, 
           'LDW': 5, 
           'EYE_CLOSED': 4, 
           'LDW_R': 5, 
           'LDW_L':5, 
           'FCW':6, 
           'camera cover!':0, 
           'infrared block!': 0 }
    
for each_driver in tqdm(ids):
    logFolder = f'Z:/VIDEOS/{each_driver}/Disk_files/debug/'
    for logfile in os.listdir(logFolder):
        driverid = logFolder.split('/')[2]
        
        # if logfile.endswith('.log'):
        alarmdict, alarmtimestamp, err = extract_indices_from_log(logFolder, logfile)
        for key, value in alarmtimestamp.items():
            for ts in value:
                if key != 'NOBODY':
                    id = driverid
                    alarm_type = key
                    date = ts[:9]
                    
                    transform_dict = {'id': id, 'date': date, 'timestamp': ts, 'type': mapping[alarm_type]}
                    jsonfile.append(transform_dict)

with open('./Datafiles/Timestamps_data.json', 'w') as jsonf:
    json.dump(jsonfile, jsonf)    
        
        

In [ ]:
#  Formatting the data to used in excel reprots
import json

with open('data_storage.json', 'r') as f:
    data = json.load(f)

for each_item in data['data']:
    each_item.update

In [ ]:
def validation_of_structure(src, participant_id):
    
    if os.path.exists(src+'/Video/827000'):
        print(f'Folder Exists 827000 in {participant_id}')
        # preparing_folder(src+f'/Video/827000')
        
    elif os.path.exists(src+'/Video/984'):
        print(f'Folder Exists 984 in {participant_id}')
        # preparing_folder(src+f'/Video/984')
    else:
        return False
    
All_drivers = os.listdir('Z:/VIDEOS/')
for each_driver in All_drivers:
    src = f'Z:/VIDEOS/{each_driver}'
    validation_of_structure(src, each_driver)

### Uploading Data to HPC Servers

In [ ]:
import shutil
import os
import tqdm
import glob
from distutils.dir_util import copy_tree



# def transferFilesToServer(src, dest):
#     # files = glob.glob(src, recursive=True)
#     # for root, dirs, files in os.walk(src):
#     #     root = "/".join(root.split('\\'))
#     #     after_root = "/".join(root.split('/')[3:])
#     #     for each_dir in dirs:
#     #         if os.path.exists(dest+after_root+each_dir):
#     #             continue
#     #         else:
#     #             os.umask(0)
                
#     #             os.mkdir(dest+after_root+'/'+each_dir, mode=0o777)
        
#     #     for file in files:
#     #         shutil.copy(root+'/'+file, dest+after_root+'/')
        
#     return f'Total File : {print(len(os.listdir(dest)))}'

def transferFilesToServer(src, dst):
    print('Started copying the files...')
    copy_tree(src, dst)
    return 'All files copied successfully'

# No '/' at end of src
def validation_of_structure(src, participant_id):
    
    if os.path.exists(src+'/Video/827000'):
        print(f'Folder Exists 827000')
        preparing_folder(src+f'/Video/827000')
        
        os.mkdir(f'K:/Backup/{participant_id}')
    elif os.path.exists(src+'/Video/984'):
        print(f'Folder Exists 984')
        preparing_folder(src+f'/Video/984')
    else:
        return False

def preparing_folder(src):
    print(f'Formatting the folder structure for Source: {src}')
    
    dest = "/".join(src.split('/')[:-1])
    print(f'Destination Folder: {dest}')
    
    copy_tree(src, dest)
    
    # make backup and transfer files
    copy_tree(src, 'K:/Backup/')
    print(f'Completed the formatting of folder structure for {src}')

# src = 'K:/Videos/1040/'
# dest = 'K:/Z/1122/'

source = 'K:/Videos/1001'
destination = 'Z:/1001'
participant_id = '1001'
 
#  this restructure the folders to same as on server
# 984/827000 still exists - delete manually
validation_of_structure(source, participant_id)

# transfer the file to hpc server
transferFilesToServer(source, destination)

In [ ]:
# os.path.exists('K:/Z/1022/Video/2021-11-06/T084931000000.asf')
# copy_tree(source, destination,)

shutil.rmtree('k:/Z/1122/Video/2021-11-02')

### Splitting of Driver files


In [ ]:
import json
def gender_stats_from_txtFile(file):
    gender_list = {'male': 0, 'female':0, 'None':0}
    with open(file) as f:
        list_of_files = json.load(f)
    for file in list_of_files:
        for file, gender in file.items():
            gender_list[gender] += 1
    # print(f"Total Stats: {gender_list}")
    return gender_list

# g_rest = gender_stats_from_txtFile('gender_deepface_1013_1014.json')
# g_1000 = gender_stats_from_txtFile('gender_deepface_1013_1014.txt')

gender_stats_from_txtFile('gender_1013_1014.txt')



In [ ]:
# Combine two json files and save it to a new file
def combine_two_list(list1, list2):
    return list1 + list2

with open('gender_deepface_1013_1014.json') as f1:
    l1 = json.load(f1)


with open('gender_deepface_1013_1014.txt') as f2:
    l2 = json.load(f2)

combined_list = combine_two_list(l1, l2)

with open('gender_1013_1014.txt', 'w') as f:
    json.dump(combined_list, f)
    

In [ ]:
import glob
from tinytag import TinyTag
path = 'Z:/VIDEOS/1013_male/Video/'
length = len(glob.glob(path+'**/*000.asf'))


def get_meta(path):
    
    # path = os.path.join(path, file)
    video = TinyTag.get(path)
    video_dict = {"album" : video.album,
                  "albumartist": video.albumartist,
                  "artist" : video.artist,
                  "aurdio_offset": video.audio_offset,
                  "bitrate" : video.bitrate,
                  "comment" : video.comment,
                  "composer" : video.composer,
                  "disc" : video.disc,
                  "disc_total" : video.disc_total,
                  "duration" : video.duration,
                  "filesize" : video.filesize,
                  "genre" :video.genre,
                  "samplerate" : video.samplerate,
                  "title" : video.title,
                  "track" : video.track,
                  "track_total" : video.track_total,
                  "year" : video.year}

    # with open('indices.json', 'w') as json_file:
    #     json.dump(dict(video_dict), json_file)
    return video_dict

def get_duration_modified(path):
    f_dict = {}
    # Glob - getting list of files
    counter = 0
    total_fileList = len(glob.glob(path+'**/*000.asf'))
    
    for file in glob.glob(path+'**/*000.asf'):
        dur = 0
        file = file.replace('\\', '/')
        metas = get_meta(file)
        try:
            if metas['duration'] < 10000:
                # print(f'Duration: {metas["duration"]}')
                dur += int(metas['duration'])
        except:
            dur += 0
            print(f'get_duration()::Error During Processing: {file}')
        dir_split = "".join(file.split("/")[-1].split("-"))
        print(f'Processing : {counter}/{total_fileList}, File: {dir_split}')
        counter += 1
        print(f_dict)
        f_dict[dir_split] = dur
    
    return f_dict

get_duration_modified(path)



In [ ]:
import pandas as pd
telematic_data = pd.read_excel("./Data _for_Integration.xlsx")
telematic_data.head()

In [ ]:
for row in telematic_data.iterrows():
    print(row[1]['PID'])
    print(row[1]['trip start'])
    print(row[1]['trip end'])

In [ ]:
telematic_data

In [ ]:
telematic_data.__len__()

In [ ]:
import pandas as pd

# data = [
#     {"id": "1001", "day": "20210920", "duration": 35.86, "yawn": {"total": 0, "timestamp": []}, "smoking": {"total": 1, "timestamp": ["20210920103721"]}, "mobilephone": {"total": 0, "timestamp": []}, "distraction": {"total": 0, "timestamp": []}, "eyeclosing": {"total": 0, "timestamp": []}, "crossinglane": {"total": 0, "timestamp": []}, "nearcollision": {"total": 0, "timestamp": []}, "stopsign": {"total": 0, "timestamp": []}, "redlight": {"total": 0, "timestamp": []}, "pedestrian": {"total": 0, "timestamp": []}},
#     {"id": "1001", "day": "20210921", "duration": 39.76, "yawn": {"total": 0, "timestamp": []}, "smoking": {"total": 1, "timestamp": ["20210921103001"]}, "mobilephone": {"total": 0, "timestamp": []}, "distraction": {"total": 0, "timestamp": []}, "eyeclosing": {"total": 0, "timestamp": []}, "crossinglane": {"total": 0, "timestamp": []}, "nearcollision": {"total": 0, "timestamp": []}, "stopsign": {"total": 0, "timestamp": []}, "redlight": {"total": 0, "timestamp": []}, "pedestrian": {"total": 0, "timestamp": []}},
#     {"id": "1001", "day": "20230401", "duration": 71.53, "yawn": {"total": 0, "timestamp": []}, "smoking": {"total": 14, "timestamp": ["20230401122415", "20230401122734", "20230401123008", "20230401123037", "20230401123201", "20230401124432", "20230401125501", "20230401183050", "20230401183141", "20230401183320", "20230401183333", "20230401184319", "20230401184722", "20230401190215"]}, "mobilephone": {"total": 0, "timestamp": []}, "distraction": {"total": 0, "timestamp": []}, "eyeclosing": {"total": 1, "timestamp": ["20230401185357"]}, "crossinglane": {"total": 12, "timestamp": ["20230401124329", "20230401124528", "20230401124012", "20230401125254", "20230401183051", "20230401183728", "20230401183837", "20230401185150", "20230401185438", "20230401185453", "20230401183557", "20230401185005"]}, "nearcollision": {"total": 0, "timestamp": []}, "stopsign": {"total": 0, "timestamp": []}, "redlight": {"total": 0, "timestamp": []}, "pedestrian": {"total": 0, "timestamp": []}}
# ]
with open('data_storage.json') as f:
    data = json.load(f)

flat_data = []

for entry in data['data']:
    id_value = entry["id"]
    day = entry["day"]
    for key, value in entry.items():
        if key not in ["id", "day", "duration"]:
            if isinstance(value, dict):
                total = value["total"]
                timestamps = value["timestamp"]
                for timestamp in timestamps:
                    flat_data.append({"id": id_value, "day": day, "type": key, "timestamp": timestamp})

df = pd.DataFrame(flat_data)
print(df)


In [ ]:
df.to_excel('video_data.xlsx', index=False)

In [ ]:
import pandas as pd
import datetime
# Load telematics data
telematics_data = pd.read_excel('Data_for_Integration.xlsx')

# Convert trip start and trip end columns to datetime
telematics_data['trip start'] = pd.to_datetime(telematics_data['trip start'])
telematics_data['trip end'] = pd.to_datetime(telematics_data['trip end'])

# Load video data from Excel file
video_data = pd.read_excel('video_data.xlsx')

# Convert the 'timestamp' column to datetime from the 'yyyymmddhhmmss' format
video_data['timestamp'] = pd.to_datetime(video_data['timestamp'], format='%Y%m%d%H%M%S')

# Merge based on PID, day, and timestamp
merged_data = pd.merge(telematics_data, video_data, how='left', left_on=['PID', 'trip start', 'trip end'], right_on=['id', 'day', 'timestamp'])

# Drop redundant columns
merged_data = merged_data.drop(['id', 'day', 'timestamp'], axis=1)

# Save the merged data to a new CSV file
merged_data.to_csv('merged_data.csv', index=False)


In [ ]:
import pandas as pd

# Load telematics data
telematics_data = pd.read_excel('Data_for_Integration.xlsx')

# Convert trip start and trip end columns to datetime
telematics_data['trip start'] = pd.to_datetime(telematics_data['trip start'])
telematics_data['trip end'] = pd.to_datetime(telematics_data['trip end'])

# Load video data from Excel file
video_data = pd.read_excel('video_data.xlsx')

# Convert the 'timestamp' column to datetime from the 'yyyymmddhhmmss' format
video_data['timestamp'] = pd.to_datetime(video_data['timestamp'], format='%Y%m%d%H%M%S')

# Merge based on 'id' and 'day'
merged_data = pd.merge(telematics_data, video_data, how='left', left_on=['PID', 'trip start', 'trip end'], right_on=['id', 'day'])

# Filter the merged data based on timestamp
merged_data = merged_data[(merged_data['timestamp'] >= merged_data['trip start']) & (merged_data['timestamp'] <= merged_data['trip end'])]

# Drop redundant columns
merged_data = merged_data.drop(['timestamp', 'day', 'id'], axis=1)

# Save the merged data to a new CSV file
merged_data.to_csv('merged_data.csv', index=False)




In [ ]:
telematics_data = pd.read_excel('Data_for_Integration.xlsx')
# Convert 'trip start' and 'trip end' columns to datetime
telematics_data['trip start'] = pd.to_datetime(telematics_data['trip start'])
telematics_data['trip end'] = pd.to_datetime(telematics_data['trip end'])

# Convert to the desired format
telematics_data['trip start formatted'] = telematics_data['trip start'].apply(lambda x: x.strftime('%Y%m%d%H%M%S'))
telematics_data['trip end formatted'] = telematics_data['trip end'].apply(lambda x: x.strftime('%Y%m%d%H%M%S'))



In [ ]:
telematics_data.head()

In [ ]:
telematics_data.to_excel('telematic_data_formatted.xlsx', index=False)

In [ ]:
# Load video data from Excel file
video_data = pd.read_excel('video_data.xlsx')

# Convert the 'timestamp' column to datetime from the 'yyyymmddhhmmss' format
video_data['timestamp'] = pd.to_datetime(video_data['timestamp'], format='%Y%m%d%H%M%S')

# Merge based on 'id' and 'day'
merged_data = pd.merge(telematics_data, video_data, how='left', left_on=['PID', 'trip start formatted', 'trip end formatted'], right_on=['id', 'day', 'timestamp'])

# Filter the merged data based on timestamp
filtered_data = merged_data[(merged_data['timestamp'] >= merged_data['trip start']) & (merged_data['timestamp'] <= merged_data['trip end'])]

# Aggregate data (you can use your own aggregation functions)
aggregated_data = filtered_data.groupby(['PID', 'trip start', 'trip end'], as_index=False).agg({
    'smoking': 'sum',
    'mobilephone': 'sum',
    # Add other columns and aggregation functions as needed
})

# # Merge the aggregated data back to telematics_data
# final_data = pd.merge(telematics_data, aggregated_data, how='left', on=['PID', 'trip start', 'trip end'])

# # Save the final merged data to a new CSV file
# final_data.to_csv('final_merged_data.csv', index=False)

In [ ]:
aggregated_data.head()

In [ ]:
import pandas as pd

# Read the CSV files
video_data = pd.read_excel('video_data.xlsx')
trip_data = pd.read_excel('telematic_data_formatted.xlsx')

# Convert timestamp columns to datetime format
video_data['timestamp'] = pd.to_datetime(video_data['timestamp'], format='%Y%m%d%H%M%S')
trip_data['trip start formatted'] = pd.to_datetime(trip_data['trip start formatted'], format='%Y%m%d%H%M%S')
trip_data['trip end formatted'] = pd.to_datetime(trip_data['trip end formatted'], format='%Y%m%d%H%M%S')

# Create a function to check if a timestamp is within a given interval
def check_interval(row):
    return video_data[(video_data['timestamp'] >= row['trip start formatted']) & (video_data['timestamp'] <= row['trip end formatted'])].shape[0]

# Apply the function to create a new column in trip_data
trip_data['count_per_type'] = trip_data.apply(check_interval, axis=1)

# Merge the two datasets based on 'id'
merged_data = pd.merge(video_data, trip_data, left_on='id', right_on='PID')


merged_data.head()
# Save the merged data to a new CSV file
merged_data.to_csv('merged_data2.csv', index=False)

In [ ]:
import pandas as pd

# Load telematic and video data
telematic_data = pd.read_csv("telematicdatafile.csv")
video_data = pd.read_csv("videofiledata.csv")

# Merge data frames
merged_data = pd.merge(telematic_data, video_data, left_on=['trip start formatted', 'trip end formatted'], right_on=['timestamp'])

# Convert timestamps to datetime format
merged_data['trip_start'] = pd.to_datetime(merged_data['trip start formatted'], format='%Y%m%d%H%M%S')
merged_data['trip_end'] = pd.to_datetime(merged_data['trip end formatted'], format='%Y%m%d%H%M%S')
merged_data['timestamp'] = pd.to_datetime(merged_data['timestamp'], format='%Y%m%d%H%M%S')

# Function to count occurrences within a given interval
def count_occurrences(row):
    interval_data = merged_data[(merged_data['timestamp'] >= row['trip_start']) & (merged_data['timestamp'] <= row['trip_end'])]
    occurrences = interval_data['type'].value_counts()
    return occurrences

# Apply the function to create new columns
occurrence_columns = ['nearcollision', 'crossinglane', 'smoking']  # Add more types as needed
for col in occurrence_columns:
    telematic_data[col + '_count'] = telematic_data.apply(count_occurrences, axis=1)[col]

# Save the updated telematic data
# telematic_data.to_csv("updated_telematic_data.csv", index=False)


In [ ]:
trip_data.head()

In [121]:
import pandas as pd

# Read the CSV files
video_data = pd.read_excel('video_data.xlsx')
trip_data = pd.read_excel('telematic_data_formatted.xlsx')

# Convert timestamp columns to datetime format
video_data['timestamp'] = pd.to_datetime(video_data['timestamp'], format='%Y%m%d%H%M%S')
trip_data['trip start formatted'] = pd.to_datetime(trip_data['trip start formatted'], format='%Y%m%d%H%M%S')
trip_data['trip end formatted'] = pd.to_datetime(trip_data['trip end formatted'], format='%Y%m%d%H%M%S')

# Create a function to check if a timestamp is within a given interval
def check_interval(row):
    total = video_data[(video_data['timestamp'] >= row['trip start formatted']) & (video_data['timestamp'] <= row['trip end formatted'])]
    # return total.shape[0]
    return total['type'].value_counts().to_dict()

# Apply the function to create a new column in trip_data
trip_data['count_per_type'] = trip_data.apply(check_interval, axis=1)

# print(trip_data.head())

In [123]:
trip_data.head()

,PID,trip start,trip end,travel_time,dist_per_trip,n_ACC,n_BRK,n_HTURN,speed_mean,speed_sd,...,air_max,fuel_mean,fuel_sd,fuel_max,thro_mean,thro_sd,thro_max,trip start formatted,trip end formatted,count_per_type
0,1009 1010,2022-06-01 13:29:00,2022-06-01 13:52:00,1406.443518,15.021497,3,3,7,41.478261,34.664456,...,36.0,69.874453,6.067238,87.843137,21.125320,11.471878,60,2022-06-01 13:29:00,2022-06-01 13:52:00,"{'crossinglane': 5, 'mobilephone': 4, 'nearcol..."
1,1009 1010,2022-06-03 14:05:00,2022-06-03 14:13:00,534.734225,2.980678,2,0,3,19.875000,26.346252,...,25.0,65.926748,5.411885,75.686275,17.794118,3.636336,23.921569,2022-06-03 14:05:00,2022-06-03 14:13:00,"{'crossinglane': 4, 'smoking': 1, 'eyeclosing'..."
2,1009 1010,2022-06-03 14:48:00,2022-06-03 14:56:00,455.779298,3.151665,1,1,3,22.500000,25.332080,...,24.0,67.816993,6.458640,80.784314,17.254902,2.780879,21.176471,2022-06-03 14:48:00,2022-06-03 14:56:00,"{'crossinglane': 37, 'eyeclosing': 6, 'nearcol..."
3,1009 1010,2022-06-06 09:42:00,2022-06-06 09:48:00,376.179314,2.388888,1,1,2,25.166667,39.050822,...,33.0,65.180599,3.986904,73.725490,17.647059,3.166530,22.745098,2022-06-06 09:42:00,2022-06-06 09:48:00,"{'crossinglane': 2, 'smoking': 1}"
4,1009 1010,2022-06-06 10:33:00,2022-06-06 10:42:00,560.031980,2.940182,2,1,2,10.600000,18.875028,...,38.0,66.345811,5.569042,81.960784,16.431373,1.871154,18.431373,2022-06-06 10:33:00,2022-06-06 10:42:00,{}


In [ ]:
import pandas as pd

# Assuming trip_data is your telematic data DataFrame
# Load telematic and video data into DataFrames
# Read the CSV files
video_data = pd.read_excel('video_data.xlsx')
trip_data = pd.read_excel('telematic_data_formatted.xlsx')

# Merge DataFrames based on timestamp
merged_data = pd.merge(telematic_data, video_data, left_on="trip start formatted", right_on="timestamp", how="left")

# Create a function to check if a timestamp is within a given interval
def check_interval(row):
    total = video_data[(video_data['timestamp'] >= row['trip start formatted']) & (video_data['timestamp'] <= row['trip end formatted'])]
    return total['type'].value_counts().to_dict()

# Apply the function to create a new column in telematic_data
telematic_data['count_per_type'] = telematic_data.apply(check_interval, axis=1)

# Expand the dictionary into separate columns
count_per_type_df = telematic_data['count_per_type'].apply(pd.Series)

# Merge the expanded data back to the original telematic DataFrame
final_data = pd.concat([telematic_data, count_per_type_df], axis=1)

# Drop the original 'count_per_type' column if needed
final_data = final_data.drop('count_per_type', axis=1)

# Fill NaN values with 0
final_data = final_data.fillna(0)

# Print or use the final DataFrame as needed
print(final_data)

pd.to_excel('final_data.xlsx', index=False)

In [141]:
import pandas as pd

# Read the CSV files
video_data = pd.read_csv('video_csv.csv')
trip_data = pd.read_csv('telematics_csv.csv')

# Convert timestamp columns to datetime format
# video_data['timestamp'] = pd.to_datetime(video_data['timestamp'], format='%Y%m%d%H%M%S')
# trip_data['trip start formatted'] = pd.to_datetime(trip_data['trip start formatted'], format='%Y%m%d%H%M%S')
# trip_data['trip end formatted'] = pd.to_datetime(trip_data['trip end formatted'], format='%Y%m%d%H%M%S')

# Create a function to check if a timestamp is within a given interval
def check_interval(row):
    total = video_data[(video_data['id'] == row['PID']) & (video_data['timestamp'] >= row['trip start formatted']) & (video_data['timestamp'] <= row['trip end formatted'])]
    # print(total['type'].value_counts())
    # print(total['type'].value_counts().to_dict())
    return total['type'].value_counts().to_dict()

# Apply the function to create a new column in trip_data
trip_data['count_per_type'] = trip_data.apply(check_interval, axis=1)
trip_data.head()

/Users/admin/.pyenv/versions/3.7.16/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,PID,trip start,trip end,travel_time,dist_per_trip,n_ACC,n_BRK,n_HTURN,speed_mean,speed_sd,...,air_max,fuel_mean,fuel_sd,fuel_max,thro_mean,thro_sd,thro_max,trip start formatted,trip end formatted,count_per_type
0,1009 1010,2022-06-01 13:29:00,2022-06-01 13:52:00,1406.443518,15.021497,3,3,7,41.478261,34.664456,...,36.0,69.874453,6.067238,87.843137,21.125320,11.471878,60,20220601132900,20220601135200,{}
1,1009 1010,2022-06-03 14:05:00,2022-06-03 14:13:00,534.734225,2.980678,2,0,3,19.875000,26.346252,...,25.0,65.926748,5.411885,75.686275,17.794118,3.636336,23.921569,20220603140500,20220603141300,{}
2,1009 1010,2022-06-03 14:48:00,2022-06-03 14:56:00,455.779298,3.151665,1,1,3,22.500000,25.332080,...,24.0,67.816993,6.458640,80.784314,17.254902,2.780879,21.176471,20220603144800,20220603145600,{}
3,1009 1010,2022-06-06 09:42:00,2022-06-06 09:48:00,376.179314,2.388888,1,1,2,25.166667,39.050822,...,33.0,65.180599,3.986904,73.725490,17.647059,3.166530,22.745098,20220606094200,20220606094800,{}
4,1009 1010,2022-06-06 10:33:00,2022-06-06 10:42:00,560.031980,2.940182,2,1,2,10.600000,18.875028,...,38.0,66.345811,5.569042,81.960784,16.431373,1.871154,18.431373,20220606103300,20220606104200,{}


In [143]:
trip_data.to_csv('trip_data.csv', index=False)

In [136]:
import pandas as pd

# Read the CSV files
video_data = pd.read_excel('video_data.xlsx')
trip_data = pd.read_excel('telematic_data_formatted.xlsx')

# Convert timestamp columns to datetime format
video_data['timestamp'] = pd.to_datetime(video_data['timestamp'], format='%Y%m%d%H%M%S')
trip_data['trip start formatted'] = pd.to_datetime(trip_data['trip start formatted'], format='%Y%m%d%H%M%S')
trip_data['trip end formatted'] = pd.to_datetime(trip_data['trip end formatted'], format='%Y%m%d%H%M%S')

# Create a function to check if a timestamp is within a given interval
def check_interval(row):
    total = video_data[(video_data['id'] == row['PID']) & (video_data['timestamp'] >= row['trip start formatted']) & (video_data['timestamp'] <= row['trip end formatted'])]
    # print(total['type'].value_counts())
    # print(total['type'].value_counts().to_dict())
    return total['type'].value_counts().to_dict()

# Apply the function to create a new column in trip_data
trip_data['count_per_type'] = trip_data.apply(check_interval, axis=1)
trip_data.head()

,PID,trip start,trip end,travel_time,dist_per_trip,n_ACC,n_BRK,n_HTURN,speed_mean,speed_sd,...,air_max,fuel_mean,fuel_sd,fuel_max,thro_mean,thro_sd,thro_max,trip start formatted,trip end formatted,count_per_type
0,1009 1010,2022-06-01 13:29:00,2022-06-01 13:52:00,1406.443518,15.021497,3,3,7,41.478261,34.664456,...,36.0,69.874453,6.067238,87.843137,21.125320,11.471878,60,2022-06-01 13:29:00,2022-06-01 13:52:00,{}
1,1009 1010,2022-06-03 14:05:00,2022-06-03 14:13:00,534.734225,2.980678,2,0,3,19.875000,26.346252,...,25.0,65.926748,5.411885,75.686275,17.794118,3.636336,23.921569,2022-06-03 14:05:00,2022-06-03 14:13:00,{}
2,1009 1010,2022-06-03 14:48:00,2022-06-03 14:56:00,455.779298,3.151665,1,1,3,22.500000,25.332080,...,24.0,67.816993,6.458640,80.784314,17.254902,2.780879,21.176471,2022-06-03 14:48:00,2022-06-03 14:56:00,{}
3,1009 1010,2022-06-06 09:42:00,2022-06-06 09:48:00,376.179314,2.388888,1,1,2,25.166667,39.050822,...,33.0,65.180599,3.986904,73.725490,17.647059,3.166530,22.745098,2022-06-06 09:42:00,2022-06-06 09:48:00,{}
4,1009 1010,2022-06-06 10:33:00,2022-06-06 10:42:00,560.031980,2.940182,2,1,2,10.600000,18.875028,...,38.0,66.345811,5.569042,81.960784,16.431373,1.871154,18.431373,2022-06-06 10:33:00,2022-06-06 10:42:00,{}


In [138]:
trip_data.to_excel('final_data.xlsx', index=False)

In [140]:
pd.read_csv('telematics_csv.csv').head()

,PID,trip start,trip end,travel_time,dist_per_trip,n_ACC,n_BRK,n_HTURN,speed_mean,speed_sd,...,air_sd,air_max,fuel_mean,fuel_sd,fuel_max,thro_mean,thro_sd,thro_max,trip start formatted,trip end formatted
0,1009 1010,2022-06-01 13:29:00,2022-06-01 13:52:00,1406.443518,15.021497,3,3,7,41.478261,34.664456,...,1.278374,36.0,69.874453,6.067238,87.843137,21.125320,11.471878,60,20220601132900,20220601135200
1,1009 1010,2022-06-03 14:05:00,2022-06-03 14:13:00,534.734225,2.980678,2,0,3,19.875000,26.346252,...,0.409432,25.0,65.926748,5.411885,75.686275,17.794118,3.636336,23.921569,20220603140500,20220603141300
2,1009 1010,2022-06-03 14:48:00,2022-06-03 14:56:00,455.779298,3.151665,1,1,3,22.500000,25.332080,...,0.468179,24.0,67.816993,6.458640,80.784314,17.254902,2.780879,21.176471,20220603144800,20220603145600
3,1009 1010,2022-06-06 09:42:00,2022-06-06 09:48:00,376.179314,2.388888,1,1,2,25.166667,39.050822,...,0.780724,33.0,65.180599,3.986904,73.725490,17.647059,3.166530,22.745098,20220606094200,20220606094800
4,1009 1010,2022-06-06 10:33:00,2022-06-06 10:42:00,560.031980,2.940182,2,1,2,10.600000,18.875028,...,1.224607,38.0,66.345811,5.569042,81.960784,16.431373,1.871154,18.431373,20220606103300,20220606104200


In [3]:
import pandas as pd

# Conversion of formatted timestamps in json file 
telematics_data['trip start'] = pd.to_datetime(telematics_data['trip start'])
telematics_data['trip end'] = pd.to_datetime(telematics_data['trip end'])

# Convert to the desired format
telematics_data['trip start formatted'] = telematics_data['trip start'].apply(lambda x: x.strftime('%Y%m%d%H%M%S'))
telematics_data['trip end formatted'] = telematics_data['trip end'].apply(lambda x: x.strftime('%Y%m%d%H%M%S'))



# Assuming trip_data is your telematic data DataFrame
# Load telematic and video data into DataFrames
video_data = pd.read_csv('video_csv.csv')
trip_data = pd.read_csv('telematics_csv.csv')

# Merge DataFrames based on timestamp
merged_data = pd.merge(trip_data, video_data, left_on="trip start formatted", right_on="timestamp", how="left")

# Create a function to check if a timestamp is within a given interval
def check_interval(row):
    total = video_data[(video_data['id'] == row['PID']) & (video_data['timestamp'] >= row['trip start formatted']) & (video_data['timestamp'] <= row['trip end formatted'])]
    return total['type'].value_counts().to_dict()


# Apply the function to create a new column in telematic_data
trip_data['count_per_type'] = trip_data.apply(check_interval, axis=1)

# Expand the dictionary into separate columns
count_per_type_df = trip_data['count_per_type'].apply(pd.Series)

# Merge the expanded data back to the original telematic DataFrame
final_data = pd.concat([trip_data, count_per_type_df], axis=1)

# Drop the original 'count_per_type' column if needed
final_data = final_data.drop('count_per_type', axis=1)

# Fill NaN values with 0
final_data = final_data.fillna(0)

# Print or use the final DataFrame as needed
final_data.to_csv('Merged_data.csv', index=False)
